# Lending Tree Loan Status - GridSearchCV

# Set Up Environment for Spark

## Colab

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Spark/

/content/drive/MyDrive/Spark


In [ ]:
# Set up environment for Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz

In [ ]:
!tar xf spark-3.3.0-bin-hadoop3.tgz

In [ ]:
# Set your spark folder to your system path environment. 
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/drive/MyDrive/Spark/spark-3.3.0-bin-hadoop3'

In [ ]:
# Install findspark using pip
!pip install -q findspark
!pip install -U pyspark==3.3
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 63.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=2ac9238506d28218a2c824c8c7bc44df580e10a8b1f116a72abe92b2f2219e0d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
# Pyspark Session for Colab
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('Colab')\
        .config('spark.driver.memory', '24g')\
        .config('spark.executor.pyspark.memory', '18g')\
        .config('spark.executor.cores', '4')\
        .config('spark.python.worker.memory', '18g')\
        .config('spark.sql.execution.arrow.pyspark.enabled', 'True')\
        .config('spark.sql.debug.maxToStringFields', '1000')\
        .config('spark.sql.autoBroadcastJoinThreshold', '-1')\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

## Paperspace Gradient

In [ ]:
# Install Java JRE/JDK in terminal
apt update
apt install default-jre
apt install default-jdk

In [ ]:
# Install findspark using pip
!pip install -q findspark
!pip install -U pyspark==3.3
import findspark
findspark.init()

In [ ]:
# Pyspark Session for Paperspace with more RAM & CPU cores available
# driver.memory=35g & executor.pyspark.memory=28 for some
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('Paperspace')\
        .config('spark.driver.memory', '38g')\
        .config('spark.executor.pyspark.memory', '32g')\
        .config('spark.executor.cores', '4')\
        .config('spark.python.worker.memory', '32g')\
        .config('spark.sql.execution.arrow.pyspark.enabled', 'True')\
        .config('spark.sql.debug.maxToStringFields', '1000')\
        .config('spark.sql.autoBroadcastJoinThreshold', '-1')\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/16 15:32:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Remove warnings
spark.sparkContext.setLogLevel('ERROR')

In [ ]:
!pip install --upgrade mlflow 
!pip install hyperopt
import random
import warnings
import numpy as np
from pyspark.sql.functions import col, round
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, LinearSVC
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
import time
from datetime import datetime, timedelta
from timeit import default_timer as timer
try:
  import mlflow.pyspark.ml
  mlflow.pyspark.ml.autolog()
except:
  print(f'Your version of MLflow ({mlflow.__version__}) does not support pyspark.ml for autologging. To use autologging, upgrade your MLflow client version or use Databricks Runtime for ML 8.3 or above.')
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.9 MB 348 kB/s 
     |████████████████████████████████| 79 kB 6.4 MB/s 
     |████████████████████████████████| 209 kB 33.5 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 182 kB 42.2 MB/s 
     |████████████████████████████████| 147 kB 40.0 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
     |████████████████████████████████| 55 kB 548 kB/s 
     |████████████████████████████████| 140 kB 53.8 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 59 kB 4.5 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.17.3-py3-none-any.whl size=139102 sha256=066f89bd942bbcdc0d99748e1b3006319e6dc9230f001dafd8def303f331cb6a
  Stored in directory: /root/.cache/pip/wheels/3f/7

In [ ]:
# Set seed 
seed_value = 42
os.environ['SparkML_HPO'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

# Upsampling - Oversample Minority Class 

## Read Data and View Schema

In [ ]:
trainDF = spark.read.csv('/content/drive/MyDrive/LoanStatus/Data/trainDF_US.csv',
                         header=True, inferSchema=True).cache()
print('\nTrain Schema')
trainDF.printSchema()

testDF = spark.read.csv('/content/drive/MyDrive/LoanStatus/Data/testDF_US.csv',
                        header=True, inferSchema=True).cache()
print('\nTest Schema')
testDF.printSchema()


Train Schema
root
 |-- loan_amnt: integer (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- out_prncp: double (nullable = true)
 |-- total_pymnt: double (nullable = true)
 |-- total_rec_int: double (nullable = true)
 |-- total_rec_late_fee: double (nullable = true)
 |-- recoveries: double (nullable = true)
 |-- last_pymnt_amnt: double (nullable = true)
 |-- collections_12_mths_ex_med: double (nullable = true)
 |-- acc_open_past_24mths: double (nullable = true)
 |-- bc_open_to_buy: double (nullable = true)
 |-- chargeoff_within_12_mths: double (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- mths_since_recent_bc: double (nullable = true)
 |-- num_bc_sats: double (nullable = true)
 |-- num_bc_tl: double (nullable = true)
 |-- num_sats: double (nulla

## Set up Vector Assembler, Scalers and Evaluators

In [ ]:
# Define features and label for train data
features = trainDF.columns[0: len(trainDF.columns) - 1]
trainDF = trainDF.select(col('loan_status').alias('label'), *features)

In [ ]:
# VectorAssembler 
vecAssembler = VectorAssembler(inputCols=features, 
                               outputCol='unscaledFeatures', 
                               handleInvalid='skip')  

# Transform train data
trainDF = vecAssembler.transform(trainDF)  

In [ ]:
# Define features and label for test data 
features = testDF.columns[0: len(testDF.columns) - 1]
testDF = testDF.select(col('loan_status').alias('label'), *features)

# Transform test data
testDF = vecAssembler.transform(testDF)  

In [ ]:
# MinMaxScaler
mmScaler = MinMaxScaler(inputCol='unscaledFeatures', 
                        outputCol='scaledFeatures',
                        min=0, max=1)

In [ ]:
# Standard scaler
stdScaler = StandardScaler(inputCol='unscaledFeatures', 
                           outputCol='scaledFeatures', 
                           withStd=True, 
                           withMean=False)

In [ ]:
# Define model evaluation
evaluator_auroc = BinaryClassificationEvaluator(labelCol='label', 
                                                metricName='areaUnderROC')

evaluator_acc = MulticlassClassificationEvaluator(labelCol='label', 
                                                  metricName='accuracy')

In [ ]:
# Change directory to write mlruns
%cd /content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/

/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV


## Grid Search

### Logistic Regression 

In [ ]:
# Set up pipeline
model = LogisticRegression(labelCol='label', 
                           featuresCol='scaledFeatures', 
                           family='binomial')
						
pipeline_lr_hpo = Pipeline(stages=[stdScaler, model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder()
            .addGrid(model.regParam, [0.01, 0.1, 1.0])
            .addGrid(model.elasticNetParam, [0.0, 0.5, 1.0])
            .addGrid(model.maxIter, [500, 1000])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_lr_hpo, 
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_lr_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# # Save best model
lr_grid = pipelineModel_lr_hpo.bestModel
lr_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_lr_us_hpo_grid')

2022/09/12 04:51:07 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '2c1c7f320a1d4d42aee6a1915c9651de', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/09/12 05:52:28 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.
Current mem limits: -1 of max -1

Setting mem limits to 3758096384 of max 3758096384



Time to perform GridSearchCV: 3707.951903104782


In [ ]:
# Find best parameters
print(pipelineModel_lr_hpo.getEstimatorParamMaps()[np.argmax(pipelineModel_lr_hpo.avgMetrics)])

{Param(parent='LogisticRegression_4d0a21af5fff', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_4d0a21af5fff', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0, Param(parent='LogisticRegression_4d0a21af5fff', name='maxIter', doc='max number of iterations (>= 0).'): 500}


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_lr_hpo.avgMetrics, 
         pipelineModel_lr_hpo.getEstimatorParamMaps()))

[(0.9681145771336286,
  {Param(parent='LogisticRegression_4d0a21af5fff', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
   Param(parent='LogisticRegression_4d0a21af5fff', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
   Param(parent='LogisticRegression_4d0a21af5fff', name='maxIter', doc='max number of iterations (>= 0).'): 500}),
 (0.9681141573612582,
  {Param(parent='LogisticRegression_4d0a21af5fff', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
   Param(parent='LogisticRegression_4d0a21af5fff', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
   Param(parent='LogisticRegression_4d0a21af5fff', name='maxIter', doc='max number of iterations (>= 0).'): 1000}),
 (0.9697153513752953,
  {Param(parent='LogisticRegression

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lr = pipelineModel_lr_hpo.transform(testDF) 

lr_auroc = evaluator_auroc.evaluate(prediction_lr)
print('GridSearchCV: Logistic Regression')
print('Area under ROC curve: %g' % (lr_auroc))
print('Test Error: %g ' % (1.0 - lr_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lr)) 

GridSearchCV: Logistic Regression
Area under ROC curve: 0.971642
Test Error: 0.028358 
Accuracy: 0.9740423101557785


In [ ]:
# Extract info from pipeline
pipelineModel_lr_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='LogisticRegression_4d0a21af5fff', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_4d0a21af5fff', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
 Param(parent='LogisticRegression_4d0a21af5fff', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'binomial',
 Param(parent='LogisticRegression_4d0a21af5fff', name='featuresCol', doc='features column name.'): 'scaledFeatures',
 Param(parent='LogisticRegression_4d0a21af5fff', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_4d0a21af5fff', name='labelCol', doc='label column name.'): 'label',
 Param(parent='LogisticRegression_4d0a21af5fff', name='maxBlockSizeInMB', doc='maximum memory i

In [ ]:
# Find best model and params
bestPipeline = pipelineModel_lr_hpo
bestModel = bestPipeline.stages[-1]
bestParams = bestModel.extractParamMap()
print(bestPipeline)
print('\n')
print(bestModel)
print('\n')
print(bestParams)

PipelineModel_f4604ac309f8


LogisticRegressionModel: uid=LogisticRegression_4d0a21af5fff, numClasses=2, numFeatures=50


{Param(parent='LogisticRegression_4d0a21af5fff', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_4d0a21af5fff', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0, Param(parent='LogisticRegression_4d0a21af5fff', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'binomial', Param(parent='LogisticRegression_4d0a21af5fff', name='featuresCol', doc='features column name.'): 'scaledFeatures', Param(parent='LogisticRegression_4d0a21af5fff', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_4d0a21af5fff', name='labelCol', doc='label col

In [ ]:
# Find best model and params
best_mod = pipelineModel_lr_hpo
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v

best_reg = sane_dict['regParam']
best_elastic_net = sane_dict['elasticNetParam']
best_max_iter = sane_dict['maxIter']
print(best_reg)
print(best_elastic_net)
print(best_max_iter)

0.01
1.0
500


### LinearSVC 

In [ ]:
# Set up pipeline
model = LinearSVC(featuresCol='scaledFeatures', 
                  labelCol='label')

pipeline_lsvc_hpo = Pipeline(stages=[stdScaler, model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder()
            .addGrid(model.regParam, [0.0, 0.01, 0.1, 1.0])
            .addGrid(model.tol, [1e-7, 1e-6, 1e-5])
            .addGrid(model.maxIter, [100, 300])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_lsvc_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO 
pipelineModel_lsvc_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
lsvc_grid = pipelineModel_lsvc_hpo.bestModel
lsvc_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_lsvc_us_hpo_grid')

2022/09/12 12:38:27 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '56d154c66eee40eaadc3a1263e78b2c4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/09/12 15:21:41 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField('unscaledFeatures', VectorUDT(), True)]. Model signature is not logged.
Current mem limits: -1 of max -1

Setting mem limits to 3758096384 of max 3758096384



Time to perform GridSearchCV: 9819.757190227509


In [ ]:
# Find best parameters
print(pipelineModel_lsvc_hpo.getEstimatorParamMaps()[np.argmax(pipelineModel_lsvc_hpo.avgMetrics)])

{Param(parent='LinearSVC_c2ed4210f380', name='regParam', doc='regularization parameter (>= 0).'): 0.0, Param(parent='LinearSVC_c2ed4210f380', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-05, Param(parent='LinearSVC_c2ed4210f380', name='maxIter', doc='max number of iterations (>= 0).'): 300}


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_lsvc_hpo.avgMetrics, 
         pipelineModel_lsvc_hpo.getEstimatorParamMaps()))

[(0.9783146147640279,
  {Param(parent='LinearSVC_c2ed4210f380', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
   Param(parent='LinearSVC_c2ed4210f380', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-07,
   Param(parent='LinearSVC_c2ed4210f380', name='maxIter', doc='max number of iterations (>= 0).'): 100}),
 (0.9796118797468962,
  {Param(parent='LinearSVC_c2ed4210f380', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
   Param(parent='LinearSVC_c2ed4210f380', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-07,
   Param(parent='LinearSVC_c2ed4210f380', name='maxIter', doc='max number of iterations (>= 0).'): 300}),
 (0.9783147754918609,
  {Param(parent='LinearSVC_c2ed4210f380', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
   Param(parent='LinearSVC_c2ed4210f380', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-06,
   Param(parent='Linear

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lsvc = pipelineModel_lsvc_hpo.transform(testDF) 

lsvc_auroc = evaluator_auroc.evaluate(prediction_lsvc)
print('GridSearchCV: LinearSVC')
print('Area under ROC curve: %g' % (lsvc_auroc))
print('Test Error: %g' % (1.0 - lsvc_auroc))
print('Accuracy: ', evaluator_acc.evaluate(prediction_lsvc)) 

GridSearchCV: LinearSVC
Area under ROC curve: 0.980204
Test Error: 0.019796
Accuracy:  0.9816335355039505


In [ ]:
# Extract info from pipeline
pipelineModel_lsvc_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='LinearSVC_c2ed4210f380', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LinearSVC_c2ed4210f380', name='featuresCol', doc='features column name.'): 'scaledFeatures',
 Param(parent='LinearSVC_c2ed4210f380', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LinearSVC_c2ed4210f380', name='labelCol', doc='label column name.'): 'label',
 Param(parent='LinearSVC_c2ed4210f380', name='maxBlockSizeInMB', doc='maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0.'): 0.0,
 Param(parent='LinearSVC_c2ed4210f380', name='maxIter', doc='max number of iterations (>= 0).'): 300,
 Param(parent='LinearSVC_c2ed4210f380', name='predictionCol', doc='prediction column name.'): 'prediction',
 Param(parent

In [ ]:
# Find best model and params
bestPipeline = pipelineModel_lsvc_hpo.bestModel
bestModel = bestPipeline.stages[-1]
bestParams = bestModel.extractParamMap()
print(bestPipeline)
print('\n')
print(bestModel)
print('\n')
print(bestParams)

PipelineModel_62ae7594223e


LinearSVCModel: uid=LinearSVC_c2ed4210f380, numClasses=2, numFeatures=50


{Param(parent='LinearSVC_c2ed4210f380', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LinearSVC_c2ed4210f380', name='featuresCol', doc='features column name.'): 'scaledFeatures', Param(parent='LinearSVC_c2ed4210f380', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LinearSVC_c2ed4210f380', name='labelCol', doc='label column name.'): 'label', Param(parent='LinearSVC_c2ed4210f380', name='maxBlockSizeInMB', doc='maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0.'): 0.0, Param(parent='LinearSVC_c2ed4210f380', name='maxIter', doc='max number of iterations (>= 0).'): 300, Param(parent='LinearSV

In [ ]:
# Find best params
best_mod = pipelineModel_lsvc_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'aggregationDepth': 2,
 'featuresCol': 'scaledFeatures',
 'fitIntercept': True,
 'labelCol': 'label',
 'maxBlockSizeInMB': 0.0,
 'maxIter': 300,
 'predictionCol': 'prediction',
 'rawPredictionCol': 'rawPrediction',
 'regParam': 0.0,
 'standardization': True,
 'threshold': 0.0,
 'tol': 1e-05}

### Decision Tree 

In [ ]:
# Set up pipeline 
model = DecisionTreeClassifier(labelCol='label', 
                               featuresCol='unscaledFeatures', 
                               impurity='gini',
                               seed=seed_value)
					
pipeline_dt_hpo = Pipeline(stages=[model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder() 
            .addGrid(model.maxDepth, [5, 10, 15])
            .addGrid(model.maxBins, [32, 64])
            .addGrid(model.impurity, ['gini', 'entropy'])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_dt_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_dt_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
dt_grid = pipelineModel_dt_hpo.bestModel
dt_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_dt_us_hpo_grid')

2022/10/12 01:31:56 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ef8a1c9742674a51b03b34455b038e9e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/12 02:05:03 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to perform GridSearchCV: 2050.343174934387


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_dt_hpo.avgMetrics, 
         pipelineModel_dt_hpo.getEstimatorParamMaps()))

[(0.9151874430963741,
  {Param(parent='DecisionTreeClassifier_ac5c3fd4a7fb', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='DecisionTreeClassifier_ac5c3fd4a7fb', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32,
   Param(parent='DecisionTreeClassifier_ac5c3fd4a7fb', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}),
 (0.9405872852369355,
  {Param(parent='DecisionTreeClassifier_ac5c3fd4a7fb', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='DecisionTreeClassifier_ac5c3fd4a7fb', name='maxBins', doc='Max number of bins for discretizing

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_dt = pipelineModel_dt_hpo.transform(testDF) 

dt_auroc = evaluator_auroc.evaluate(prediction_dt)
print('GridSearchCV: Decision Tree')
print('Area under ROC curve: %g' % (dt_auroc))
print('Test Error: %g '% (1.0 - dt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_dt)) 

GridSearchCV: Decision Tree
Area under ROC curve: 0.960222
Test Error: 0.039778 
Accuracy: 0.9810508401680568


In [ ]:
# Extract info from pipeline
pipelineModel_dt_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='DecisionTreeClassifier_ac5c3fd4a7fb', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='DecisionTreeClassifier_ac5c3fd4a7fb', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='DecisionTreeClassifier_ac5c3fd4a7fb', name='featuresCol', doc='features column name.'): 'unscaledFeatures',
 Param(parent='DecisionTreeClassifier_ac5c3fd4a7fb', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini

In [ ]:
# Find best model and params
best_mod = pipelineModel_dt_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featuresCol': 'unscaledFeatures',
 'impurity': 'entropy',
 'labelCol': 'label',
 'leafCol': '',
 'maxBins': 64,
 'maxDepth': 15,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'minWeightFractionPerNode': 0.0,
 'predictionCol': 'prediction',
 'probabilityCol': 'probability',
 'rawPredictionCol': 'rawPrediction',
 'seed': 42}

#### More Params

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder() 
            .addGrid(model.maxDepth, [5, 10, 15, 20, 25, 30])
            .addGrid(model.maxBins, [16, 32, 64, 128])
            .addGrid(model.impurity, ['gini', 'entropy'])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_dt_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_dt_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
dt_grid = pipelineModel_dt_hpo.bestModel
dt_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_dt_us_hpo_grid_moreParams')

2022/10/11 20:06:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '16f3ab6981394684937bfbe077d94a3b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/12 00:39:34 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.
2022/10/12 00:40:32 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpw5ma7wul/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
2022/10/12 00:41:15 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp_7l_lw24/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to perform GridSearchCV:

In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_dt_hpo.avgMetrics, 
         pipelineModel_dt_hpo.getEstimatorParamMaps()))

[(0.9138925723080326,
  {Param(parent='DecisionTreeClassifier_a92c3021154f', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='DecisionTreeClassifier_a92c3021154f', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 16,
   Param(parent='DecisionTreeClassifier_a92c3021154f', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}),
 (0.9431497714808985,
  {Param(parent='DecisionTreeClassifier_a92c3021154f', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='DecisionTreeClassifier_a92c3021154f', name='maxBins', doc='Max number of bins for discretizing

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_dt = pipelineModel_dt_hpo.transform(testDF) 

dt_auroc = evaluator_auroc.evaluate(prediction_dt)
print('GridSearchCV: Decision Tree')
print('Area under ROC curve: %g' % (dt_auroc))
print('Test Error: %g '% (1.0 - dt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_dt)) 

GridSearchCV: Decision Tree
Area under ROC curve: 0.953074
Test Error: 0.0469264 
Accuracy: 0.9714826127873879


In [ ]:
# Extract info from pipeline
pipelineModel_dt_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='DecisionTreeClassifier_a92c3021154f', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='DecisionTreeClassifier_a92c3021154f', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='DecisionTreeClassifier_a92c3021154f', name='featuresCol', doc='features column name.'): 'unscaledFeatures',
 Param(parent='DecisionTreeClassifier_a92c3021154f', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini

In [ ]:
# Find best model and params
best_mod = pipelineModel_dt_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featuresCol': 'unscaledFeatures',
 'impurity': 'gini',
 'labelCol': 'label',
 'leafCol': '',
 'maxBins': 16,
 'maxDepth': 30,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'minWeightFractionPerNode': 0.0,
 'predictionCol': 'prediction',
 'probabilityCol': 'probability',
 'rawPredictionCol': 'rawPrediction',
 'seed': 42}

### Random Forest

In [ ]:
# Set up pipeline 
model = RandomForestClassifier(labelCol='label', 
                               featuresCol='unscaledFeatures',
                               seed=seed_value)
						
pipeline_rf_hpo = Pipeline(stages=[model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder() 
            .addGrid(model.maxDepth, [10, 15])
            .addGrid(model.maxBins, [32, 64])
            .addGrid(model.numTrees, [10, 30])
            .addGrid(model.impurity, ['gini', 'entropy'])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_rf_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO 
pipelineModel_rf_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
rf_grid = pipelineModel_rf_hpo.bestModel
rf_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_rf_us_hpo_grid')

2022/10/11 01:18:59 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6506437536984c5bad77a3908cd75cbf', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/11 12:57:24 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpemqfgmtb/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
2022/10/11 12:58:10 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp7zbm272o/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to perform GridSearchCV: 41951.0479080677


In [ ]:
# Find best parameters
print(pipelineModel_rf_hpo.getEstimatorParamMaps()[np.argmax(pipelineModel_rf_hpo.avgMetrics)])

{Param(parent='RandomForestClassifier_2e632e652457', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 15, Param(parent='RandomForestClassifier_2e632e652457', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 64, Param(parent='RandomForestClassifier_2e632e652457', name='numTrees', doc='Number of trees to train (>= 1).'): 30, Param(parent='RandomForestClassifier_2e632e652457', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_rf_hpo.avgMetrics, 
         pipelineModel_rf_hpo.getEstimatorParamMaps()))

[(0.9735197519479852,
  {Param(parent='RandomForestClassifier_2e632e652457', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 10,
   Param(parent='RandomForestClassifier_2e632e652457', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32,
   Param(parent='RandomForestClassifier_2e632e652457', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
   Param(parent='RandomForestClassifier_2e632e652457', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}),
 (0.9730370306403409,
  {Param(parent='RandomForestClassifier_2e632e652457', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_rf = pipelineModel_rf_hpo.transform(testDF) 

rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('GridSearchCV: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

GridSearchCV: Random Forest
Area under ROC curve: 0.981181
Test Error: 0.0188195 
Accuracy: 0.9791455189110072


In [ ]:
# Extract info from pipeline
pipelineModel_rf_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='RandomForestClassifier_2e632e652457', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True,
 Param(parent='RandomForestClassifier_2e632e652457', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='RandomForestClassifier_2e632e652457', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='RandomForestClassifier_2e632e652457', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supp

In [ ]:
# Find best model params
best_mod = pipelineModel_rf_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'bootstrap': True,
 'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featureSubsetStrategy': 'auto',
 'featuresCol': 'unscaledFeatures',
 'impurity': 'gini',
 'labelCol': 'label',
 'leafCol': '',
 'maxBins': 64,
 'maxDepth': 15,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'minWeightFractionPerNode': 0.0,
 'numTrees': 30,
 'predictionCol': 'prediction',
 'probabilityCol': 'probability',
 'rawPredictionCol': 'rawPrediction',
 'seed': 42,
 'subsamplingRate': 1.0}

In [ ]:
# Find best model params
best_impurity = sane_dict['impurity']
best_numTrees = sane_dict['numTrees']
best_maxBins = sane_dict['maxBins']
best_maxDepth = sane_dict['maxDepth']
print(best_impurity)
print(best_numTrees) 
print(best_maxBins)
print(best_maxDepth)

gini
30
64
15


### Gradient Boosted Trees

In [ ]:
# Set up pipeline
model = GBTClassifier(labelCol='label', 
                      featuresCol='unscaledFeatures',
                      seed=seed_value)
						
pipeline_gbt_hpo = Pipeline(stages=[model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder() 
            .addGrid(model.maxDepth, [5, 10, 15])
            .addGrid(model.maxBins, [32, 64])
            .addGrid(model.maxIter, [10])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_gbt_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_gbt_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
gbt_grid = pipelineModel_gbt_hpo.bestModel
gbt_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_gbt_us_hpo_grid')

2022/10/10 21:36:49 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '7c04b608003541febf14cb801d6090ee', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/10 23:59:39 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.
2022/10/11 00:00:33 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpzh1dpkge/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
2022/10/11 00:01:15 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpvl4elzfl/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to perform GridSearchCV:

In [ ]:
# Find best parameters
print(pipelineModel_gbt_hpo.getEstimatorParamMaps()[np.argmax(pipelineModel_gbt_hpo.avgMetrics)])

{Param(parent='GBTClassifier_41665e5b1949', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 15, Param(parent='GBTClassifier_41665e5b1949', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 64, Param(parent='GBTClassifier_41665e5b1949', name='maxIter', doc='max number of iterations (>= 0).'): 10}


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_gbt_hpo.avgMetrics, 
         pipelineModel_gbt_hpo.getEstimatorParamMaps()))

[(0.9719265974945634,
  {Param(parent='GBTClassifier_41665e5b1949', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='GBTClassifier_41665e5b1949', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32,
   Param(parent='GBTClassifier_41665e5b1949', name='maxIter', doc='max number of iterations (>= 0).'): 10}),
 (0.9719671645400161,
  {Param(parent='GBTClassifier_41665e5b1949', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='GBTClassifier_41665e5b1949', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 64,
   Param(parent='GB

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_gbt = pipelineModel_gbt_hpo.transform(testDF) 

gbt_auroc = evaluator_auroc.evaluate(prediction_gbt)
print('GridSearchCV: Gradient Boosted Trees')
print('Area under ROC curve: %g' % (gbt_auroc))
print('Test Error: %g ' % (1.0 - gbt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_gbt)) 

GridSearchCV: Gradient Boosted Trees
Area under ROC curve: 0.984932
Test Error: 0.0150683 
Accuracy: 0.9823503432584231


In [ ]:
# Extract info from pipeline
pipelineModel_gbt_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='GBTClassifier_41665e5b1949', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='GBTClassifier_41665e5b1949', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='GBTClassifier_41665e5b1949', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for r

In [ ]:
best_mod = pipelineModel_gbt_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featureSubsetStrategy': 'all',
 'featuresCol': 'unscaledFeatures',
 'impurity': 'variance',
 'labelCol': 'label',
 'leafCol': '',
 'lossType': 'logistic',
 'maxBins': 64,
 'maxDepth': 15,
 'maxIter': 10,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'minWeightFractionPerNode': 0.0,
 'predictionCol': 'prediction',
 'probabilityCol': 'probability',
 'rawPredictionCol': 'rawPrediction',
 'seed': 42,
 'stepSize': 0.1,
 'subsamplingRate': 1.0,
 'validationTol': 0.01}

# SMOTE - Split Over Upsampling 

## Read Data and View Schema

In [ ]:
trainDF = spark.read.csv('/content/drive/MyDrive/LoanStatus/Data/trainDF_SMOTE.csv',
                         header=True, inferSchema=True).cache()
trainDF = trainDF \
  .withColumn('loan_amnt', trainDF['loan_amnt'].cast(IntegerType())) \
  .withColumn('revol_bal', trainDF['revol_bal'].cast(IntegerType())) \
  .withColumn('term_ 60 months', trainDF['term_ 60 months'].cast(IntegerType())) \
  .withColumn('grade_B', trainDF['grade_B'].cast(IntegerType())) \
  .withColumn('grade_C', trainDF['grade_C'].cast(IntegerType())) \
  .withColumn('grade_D', trainDF['grade_D'].cast(IntegerType())) \
  .withColumn('home_ownership_MORTGAGE', trainDF['home_ownership_MORTGAGE'].cast(IntegerType())) \
  .withColumn('home_ownership_OWN', trainDF['home_ownership_OWN'].cast(IntegerType())) \
  .withColumn('home_ownership_RENT', trainDF['home_ownership_RENT'].cast(IntegerType())) \
  .withColumn('verification_status_Source Verified', trainDF['verification_status_Source Verified'].cast(IntegerType())) \
  .withColumn('verification_status_Verified', trainDF['verification_status_Verified'].cast(IntegerType())) \
  .withColumn('purpose_credit_card', trainDF['purpose_credit_card'].cast(IntegerType())) \
  .withColumn('initial_list_status_w', trainDF['initial_list_status_w'].cast(IntegerType())) \
  .withColumn('application_type_Joint App', trainDF['application_type_Joint App'].cast(IntegerType())) \
  .withColumn('disbursement_method_DirectPay', trainDF['disbursement_method_DirectPay'].cast(IntegerType())) 
print('\nTrain Schema')
trainDF.printSchema()

testDF = spark.read.csv('/content/drive/MyDrive/LoanStatus/Data/testDF_SMOTE.csv',
                        header=True, inferSchema=True).cache()
print('\nTest Schema')
testDF.printSchema()


Train Schema
root
 |-- loan_amnt: integer (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- out_prncp: double (nullable = true)
 |-- total_pymnt: double (nullable = true)
 |-- total_rec_int: double (nullable = true)
 |-- total_rec_late_fee: double (nullable = true)
 |-- recoveries: double (nullable = true)
 |-- last_pymnt_amnt: double (nullable = true)
 |-- collections_12_mths_ex_med: double (nullable = true)
 |-- acc_open_past_24mths: double (nullable = true)
 |-- bc_open_to_buy: double (nullable = true)
 |-- chargeoff_within_12_mths: double (nullable = true)
 |-- delinq_amnt: double (nullable = true)
 |-- mths_since_recent_bc: double (nullable = true)
 |-- num_bc_sats: double (nullable = true)
 |-- num_bc_tl: double (nullable = true)
 |-- num_sats: double (nulla

## Set up Vector Assembler

In [ ]:
# Define features and label for train data
features = trainDF.columns[0: len(trainDF.columns) - 1]
trainDF = trainDF.select(col('loan_status').alias('label'), *features)

# Transform train data
trainDF = vecAssembler.transform(trainDF)  

In [ ]:
# Define features and label for test data 
features = testDF.columns[0: len(testDF.columns) - 1]
testDF = testDF.select(col('loan_status').alias('label'), *features)

# Transform test data
testDF = vecAssembler.transform(testDF)  

In [ ]:
# Change directory to write mlruns
%cd /content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/

/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV


## Grid Search

### Logistic Regression

In [ ]:
# Set up pipeline
model = LogisticRegression(labelCol='label', 
                           featuresCol='scaledFeatures', 
                           family='binomial')
						
pipeline_lr_hpo = Pipeline(stages=[stdScaler, model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder()
            .addGrid(model.regParam, [0.01, 0.1, 1.0])
            .addGrid(model.elasticNetParam, [0.0, 0.5, 1.0])
            .addGrid(model.maxIter, [500, 1000])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_lr_hpo, 
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_lr_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
lr_grid = pipelineModel_lr_hpo.bestModel
lr_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_lr_smote_hpo_grid')

2022/09/12 15:22:59 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f3ce189f3e4f4a1b9785ff52706ead0a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
Current mem limits: -1 of max -1                                                

Setting mem limits to 3758096384 of max 3758096384



Time to perform GridSearchCV: 3538.256532430649


In [ ]:
# Find best parameters
print(pipelineModel_lr_hpo.getEstimatorParamMaps()[np.argmax(pipelineModel_lr_hpo.avgMetrics)])

{Param(parent='LogisticRegression_1a8b16aee211', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='LogisticRegression_1a8b16aee211', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0, Param(parent='LogisticRegression_1a8b16aee211', name='maxIter', doc='max number of iterations (>= 0).'): 1000}


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_lr_hpo.avgMetrics, 
         pipelineModel_lr_hpo.getEstimatorParamMaps()))

[(0.9778755360734508,
  {Param(parent='LogisticRegression_1a8b16aee211', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
   Param(parent='LogisticRegression_1a8b16aee211', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
   Param(parent='LogisticRegression_1a8b16aee211', name='maxIter', doc='max number of iterations (>= 0).'): 500}),
 (0.9778751109984445,
  {Param(parent='LogisticRegression_1a8b16aee211', name='regParam', doc='regularization parameter (>= 0).'): 0.01,
   Param(parent='LogisticRegression_1a8b16aee211', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
   Param(parent='LogisticRegression_1a8b16aee211', name='maxIter', doc='max number of iterations (>= 0).'): 1000}),
 (0.9791926748828731,
  {Param(parent='LogisticRegression

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lr = pipelineModel_lr_hpo.transform(testDF) 

lr_auroc = evaluator_auroc.evaluate(prediction_lr)
print('GridSearchCV: Logistic Regression')
print('Area under ROC curve: %g' % (lr_auroc))
print('Test Error: %g ' % (1.0 - lr_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_lr)) 

GridSearchCV: Logistic Regression
Area under ROC curve: 0.970526
Test Error: 0.0294738 
Accuracy: 0.9745810721131724


In [ ]:
# Extract info from pipeline
pipelineModel_lr_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='LogisticRegression_1a8b16aee211', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LogisticRegression_1a8b16aee211', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
 Param(parent='LogisticRegression_1a8b16aee211', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'binomial',
 Param(parent='LogisticRegression_1a8b16aee211', name='featuresCol', doc='features column name.'): 'scaledFeatures',
 Param(parent='LogisticRegression_1a8b16aee211', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LogisticRegression_1a8b16aee211', name='labelCol', doc='label column name.'): 'label',
 Param(parent='LogisticRegression_1a8b16aee211', name='maxBlockSizeInMB', doc='maximum memory i

In [ ]:
# Find best model and params
bestPipeline = pipelineModel_lr_hpo.bestModel
bestModel = bestPipeline.stages[-1]
bestParams = bestModel.extractParamMap()
print(bestPipeline)
print('\n')
print(bestModel)
print('\n')
print(bestParams)

PipelineModel_48ff2dae612f


LogisticRegressionModel: uid=LogisticRegression_1a8b16aee211, numClasses=2, numFeatures=50


{Param(parent='LogisticRegression_1a8b16aee211', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_1a8b16aee211', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0, Param(parent='LogisticRegression_1a8b16aee211', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'binomial', Param(parent='LogisticRegression_1a8b16aee211', name='featuresCol', doc='features column name.'): 'scaledFeatures', Param(parent='LogisticRegression_1a8b16aee211', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_1a8b16aee211', name='labelCol', doc='label col

In [ ]:
# Find best model and params
best_mod = pipelineModel_lr_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v

best_reg = sane_dict['regParam']
best_elastic_net = sane_dict['elasticNetParam']
best_max_iter = sane_dict['maxIter']
print(best_reg)
print(best_elastic_net)
print(best_max_iter)

0.01
1.0
1000


### LinearSVC

In [ ]:
# Set up pipeline
model = LinearSVC(featuresCol='scaledFeatures', 
                  labelCol='label')

pipeline_lsvc_hpo = Pipeline(stages=[stdScaler, model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder()
            .addGrid(model.regParam, [0.0, 0.01, 0.1, 1.0])
            .addGrid(model.tol, [1e-7, 1e-6, 1e-5])
            .addGrid(model.maxIter, [100])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_lsvc_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_lsvc_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
lsvc_grid = pipelineModel_lsvc_hpo.bestModel
lsvc_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_lsvc_smote_hpo_grid')

2022/10/10 19:20:09 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e05424308fda4d999924b85ad77994ee', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/10 20:35:23 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to perform GridSearchCV: 4582.975047588348


In [ ]:
# Find best parameters
print(pipelineModel_lsvc_hpo.getEstimatorParamMaps()[np.argmax(pipelineModel_lsvc_hpo.avgMetrics)])

{Param(parent='LinearSVC_ae6f900c2384', name='regParam', doc='regularization parameter (>= 0).'): 0.0, Param(parent='LinearSVC_ae6f900c2384', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-05, Param(parent='LinearSVC_ae6f900c2384', name='maxIter', doc='max number of iterations (>= 0).'): 100}


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_lsvc_hpo.avgMetrics, 
         pipelineModel_lsvc_hpo.getEstimatorParamMaps()))

[(0.9873127154721812,
  {Param(parent='LinearSVC_ae6f900c2384', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
   Param(parent='LinearSVC_ae6f900c2384', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-07,
   Param(parent='LinearSVC_ae6f900c2384', name='maxIter', doc='max number of iterations (>= 0).'): 100}),
 (0.9873126219826217,
  {Param(parent='LinearSVC_ae6f900c2384', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
   Param(parent='LinearSVC_ae6f900c2384', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-06,
   Param(parent='LinearSVC_ae6f900c2384', name='maxIter', doc='max number of iterations (>= 0).'): 100}),
 (0.9873127799419226,
  {Param(parent='LinearSVC_ae6f900c2384', name='regParam', doc='regularization parameter (>= 0).'): 0.0,
   Param(parent='LinearSVC_ae6f900c2384', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-05,
   Param(parent='Linear

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_lsvc = pipelineModel_lsvc_hpo.transform(testDF) 

lsvc_auroc = evaluator_auroc.evaluate(prediction_lsvc)
print('GridSearchCV: LinearSVC')
print('Area under ROC curve: %g' % (lsvc_auroc))
print('Test Error: %g' % (1.0 - lsvc_auroc))
print('Accuracy: ', evaluator_acc.evaluate(prediction_lsvc)) 

GridSearchCV: LinearSVC
Area under ROC curve: 0.979112
Test Error: 0.0208878
Accuracy:  0.9819711288334763


In [ ]:
# Extract info from pipeline
pipelineModel_lsvc_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='LinearSVC_ae6f900c2384', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2,
 Param(parent='LinearSVC_ae6f900c2384', name='featuresCol', doc='features column name.'): 'scaledFeatures',
 Param(parent='LinearSVC_ae6f900c2384', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LinearSVC_ae6f900c2384', name='labelCol', doc='label column name.'): 'label',
 Param(parent='LinearSVC_ae6f900c2384', name='maxBlockSizeInMB', doc='maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0.'): 0.0,
 Param(parent='LinearSVC_ae6f900c2384', name='maxIter', doc='max number of iterations (>= 0).'): 100,
 Param(parent='LinearSVC_ae6f900c2384', name='predictionCol', doc='prediction column name.'): 'prediction',
 Param(parent

In [ ]:
# Find best model and params
bestPipeline = pipelineModel_lsvc_hpo.bestModel
bestModel = bestPipeline.stages[-1]
bestParams = bestModel.extractParamMap()
print(bestPipeline)
print('\n')
print(bestModel)
print('\n')
print(bestParams)

PipelineModel_8276520eaaad


LinearSVCModel: uid=LinearSVC_ae6f900c2384, numClasses=2, numFeatures=50


{Param(parent='LinearSVC_ae6f900c2384', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LinearSVC_ae6f900c2384', name='featuresCol', doc='features column name.'): 'scaledFeatures', Param(parent='LinearSVC_ae6f900c2384', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LinearSVC_ae6f900c2384', name='labelCol', doc='label column name.'): 'label', Param(parent='LinearSVC_ae6f900c2384', name='maxBlockSizeInMB', doc='maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0.'): 0.0, Param(parent='LinearSVC_ae6f900c2384', name='maxIter', doc='max number of iterations (>= 0).'): 100, Param(parent='LinearSV

In [ ]:
# Find best model and params
best_mod = pipelineModel_lsvc_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'aggregationDepth': 2,
 'featuresCol': 'scaledFeatures',
 'fitIntercept': True,
 'labelCol': 'label',
 'maxBlockSizeInMB': 0.0,
 'maxIter': 100,
 'predictionCol': 'prediction',
 'rawPredictionCol': 'rawPrediction',
 'regParam': 0.0,
 'standardization': True,
 'threshold': 0.0,
 'tol': 1e-05}

### Decision Tree

In [ ]:
# Set up pipeline 
model = DecisionTreeClassifier(labelCol='label', 
                               featuresCol='unscaledFeatures', 
                               impurity='gini',
                               seed=seed_value)
						
pipeline_dt_hpo = Pipeline(stages=[model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder() 
            .addGrid(model.maxDepth, [5, 10, 15])
            .addGrid(model.maxBins, [32, 64])
            .addGrid(model.impurity, ['gini', 'entropy'])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_dt_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_dt_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
dt_grid = pipelineModel_dt_hpo.bestModel
dt_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_dt_smote_hpo_grid')

2022/10/11 00:59:52 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b7eeb1bd1b9d4290afa11dc2878b27b7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/11 01:38:32 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.


Time to perform GridSearchCV: 2388.0911955833435


In [ ]:
# Find best parameters
print(pipelineModel_dt_hpo.getEstimatorParamMaps()[np.argmax(pipelineModel_dt_hpo.avgMetrics)])

{Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 15, Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 64, Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'entropy'}


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_dt_hpo.avgMetrics, 
         pipelineModel_dt_hpo.getEstimatorParamMaps()))

[(0.9438826695906704,
  {Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32,
   Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}),
 (0.9368405569868513,
  {Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='maxBins', doc='Max number of bins for discretizing

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_dt = pipelineModel_dt_hpo.transform(testDF) 

dt_auroc = evaluator_auroc.evaluate(prediction_dt)
print('GridSearchCV: Decision Tree')
print('Area under ROC curve: %g' % (dt_auroc))
print('Test Error: %g '% (1.0 - dt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_dt)) 

GridSearchCV: Decision Tree
Area under ROC curve: 0.968558
Test Error: 0.0314419 
Accuracy: 0.9862372911141274


In [ ]:
# Extract info from pipeline
pipelineModel_dt_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='featuresCol', doc='features column name.'): 'unscaledFeatures',
 Param(parent='DecisionTreeClassifier_b7e5cdaeea9a', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini

In [ ]:
# Find best model and params
bestPipeline = pipelineModel_dt_hpo.bestModel
bestPipeline.stages 

[DecisionTreeClassificationModel: uid=DecisionTreeClassifier_b7e5cdaeea9a, depth=15, numNodes=3727, numClasses=2, numFeatures=50]

In [ ]:
# Find best model and params
best_mod = pipelineModel_dt_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featuresCol': 'unscaledFeatures',
 'impurity': 'entropy',
 'labelCol': 'label',
 'leafCol': '',
 'maxBins': 64,
 'maxDepth': 15,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'minWeightFractionPerNode': 0.0,
 'predictionCol': 'prediction',
 'probabilityCol': 'probability',
 'rawPredictionCol': 'rawPrediction',
 'seed': 42}

#### More Params

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder() 
            .addGrid(model.maxDepth, [5, 10, 15, 20, 25, 30])
            .addGrid(model.maxBins, [16, 32, 64, 128])
            .addGrid(model.impurity, ['gini', 'entropy'])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_dt_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_dt_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
dt_grid = pipelineModel_dt_hpo.bestModel
dt_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_dt_smote_hpo_grid_moreParams')

2022/10/11 21:07:26 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd7605fcb68534ac6a59132d9d74fbe3b', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/12 02:12:04 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.
Time to perform GridSearchCV: 18352.600459575653


In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_dt = pipelineModel_dt_hpo.transform(testDF) 

dt_auroc = evaluator_auroc.evaluate(prediction_dt)
print('GridSearchCV: Decision Tree')
print('Area under ROC curve: %g' % (dt_auroc))
print('Test Error: %g '% (1.0 - dt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_dt)) 

GridSearchCV: Decision Tree
Area under ROC curve: 0.959274
Test Error: 0.0407262
Accuracy: 0.9808149872940045


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_dt_hpo.avgMetrics, 
         pipelineModel_dt_hpo.getEstimatorParamMaps()))

[(0.9449530553497675,
  {Param(parent='DecisionTreeClassifier_446eea5e6566', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='DecisionTreeClassifier_446eea5e6566', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 16,
   Param(parent='DecisionTreeClassifier_446eea5e6566', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}),
 (0.9345136722725162,
  {Param(parent='DecisionTreeClassifier_446eea5e6566', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='DecisionTreeClassifier_446eea5e6566', name='maxBins', doc='Max number of bins for discretizing

In [ ]:
# Extract info from pipeline
pipelineModel_dt_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='DecisionTreeClassifier_446eea5e6566', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='DecisionTreeClassifier_446eea5e6566', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='DecisionTreeClassifier_446eea5e6566', name='featuresCol', doc='features column name.'): 'unscaledFeatures',
 Param(parent='DecisionTreeClassifier_446eea5e6566', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini

In [ ]:
# Find best model and params
best_mod = pipelineModel_dt_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featuresCol': 'unscaledFeatures',
 'impurity': 'entropy',
 'labelCol': 'label',
 'leafCol': '',
 'maxBins': 128,
 'maxDepth': 25,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'minWeightFractionPerNode': 0.0,
 'predictionCol': 'prediction',
 'probabilityCol': 'probability',
 'rawPredictionCol': 'rawPrediction',
 'seed': 42}

### Random Forest

In [ ]:
# Set up pipeline
model = RandomForestClassifier(labelCol='label', 
                               featuresCol='unscaledFeatures',
                               seed=seed_value)
						
# Set up pipeline
pipeline_rf_hpo = Pipeline(stages=[model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder() 
            .addGrid(model.maxDepth, [10, 15])
            .addGrid(model.maxBins, [32, 64])
            .addGrid(model.numTrees, [10, 30])
            .addGrid(model.impurity, ['gini', 'entropy'])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_rf_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_rf_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
rf_grid = pipelineModel_rf_hpo.bestModel
rf_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_rf_smote_hpo_grid')

2022/10/11 01:39:45 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '60f18565cfab405093e14b7c1b93532e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/11 12:48:28 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp39kul443/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
2022/10/11 12:49:15 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpce9hlzj_/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to perform GridSearchCV: 40170.12471294403


In [ ]:
# Find best parameters
print(pipelineModel_rf_hpo.getEstimatorParamMaps()[np.argmax(pipelineModel_rf_hpo.avgMetrics)])

{Param(parent='RandomForestClassifier_e535a2363763', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 15, Param(parent='RandomForestClassifier_e535a2363763', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 64, Param(parent='RandomForestClassifier_e535a2363763', name='numTrees', doc='Number of trees to train (>= 1).'): 30, Param(parent='RandomForestClassifier_e535a2363763', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_rf_hpo.avgMetrics, 
         pipelineModel_rf_hpo.getEstimatorParamMaps()))

[(0.993415747629764,
  {Param(parent='RandomForestClassifier_e535a2363763', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 10,
   Param(parent='RandomForestClassifier_e535a2363763', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32,
   Param(parent='RandomForestClassifier_e535a2363763', name='numTrees', doc='Number of trees to train (>= 1).'): 10,
   Param(parent='RandomForestClassifier_e535a2363763', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini'): 'gini'}),
 (0.9937089892299895,
  {Param(parent='RandomForestClassifier_e535a2363763', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_rf = pipelineModel_rf_hpo.transform(testDF) 

rf_auroc = evaluator_auroc.evaluate(prediction_rf)
print('GridSearchCV: Random Forest')
print('Area under ROC curve: %g' % (rf_auroc))
print('Test Error: %g ' % (1.0 - rf_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_rf)) 

GridSearchCV: Random Forest
Area under ROC curve: 0.97809
Test Error: 0.0219097 
Accuracy: 0.975866701505065


In [ ]:
# Extract info from pipeline
pipelineModel_rf_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='RandomForestClassifier_e535a2363763', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True,
 Param(parent='RandomForestClassifier_e535a2363763', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='RandomForestClassifier_e535a2363763', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='RandomForestClassifier_e535a2363763', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supp

In [ ]:
# Find best model and params
best_mod = pipelineModel_rf_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'bootstrap': True,
 'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featureSubsetStrategy': 'auto',
 'featuresCol': 'unscaledFeatures',
 'impurity': 'gini',
 'labelCol': 'label',
 'leafCol': '',
 'maxBins': 64,
 'maxDepth': 15,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'minWeightFractionPerNode': 0.0,
 'numTrees': 30,
 'predictionCol': 'prediction',
 'probabilityCol': 'probability',
 'rawPredictionCol': 'rawPrediction',
 'seed': 42,
 'subsamplingRate': 1.0}

In [ ]:
# Find best model and params
best_impurity = sane_dict['impurity']
best_numTrees = sane_dict['numTrees']
best_maxBins = sane_dict['maxBins']
best_maxDepth = sane_dict['maxDepth']
print(best_impurity)
print(best_numTrees) 
print(best_maxBins)
print(best_maxDepth)

gini
30
64
15


### Gradient Boosted Trees

In [ ]:
# Set up pipeline
model = GBTClassifier(labelCol='label', 
                      featuresCol='unscaledFeatures',
                      seed=seed_value)				

pipeline_gbt_hpo = Pipeline(stages=[model])

In [ ]:
# Create parameter grid
paramGrid = (ParamGridBuilder() 
            .addGrid(model.maxDepth, [5, 10, 15])
            .addGrid(model.maxBins, [32, 64])
            .addGrid(model.maxIter, [10])
            .build())

In [ ]:
# Define type of cross-validation
cv = CrossValidator(estimator=pipeline_gbt_hpo,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator_auroc,
                    numFolds=5, parallelism=5)

In [ ]:
# Start timer for experiment
search_time_start = time.time()

# Run HPO
pipelineModel_gbt_hpo = cv.fit(trainDF)

# End timer for experiment
print('Time to perform GridSearchCV:', time.time() - search_time_start)

# Save best model
gbt_grid = pipelineModel_gbt_hpo.bestModel
gbt_grid.save('/content/drive/MyDrive/LoanStatus/Python/Models/ML/SparkML/Models/GridSearchCV/pipelineModel_gbt_smote_hpo_grid')

2022/10/11 16:02:42 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '632d76fa07cf4f7aba9967984272045f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pyspark.ml workflow
2022/10/11 18:39:32 WARNING mlflow.pyspark.ml: Model inputs contain unsupported Spark data types: [StructField(unscaledFeatures,VectorUDT,true)]. Model signature is not logged.
022/10/11 18:40:26 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpsspix5l1/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
2022/10/11 18:41:08 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpkhylcbdh/model, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
Time to perform GridSearchCV: 

In [ ]:
# Find best parameters
print(pipelineModel_gbt_hpo.getEstimatorParamMaps()[np.argmax(pipelineModel_gbt_hpo.avgMetrics)])

{Param(parent='GBTClassifier_8c9c74b6ce2d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 15, Param(parent='GBTClassifier_8c9c74b6ce2d', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 64, Param(parent='GBTClassifier_8c9c74b6ce2d', name='maxIter', doc='max number of iterations (>= 0).'): 10}


In [ ]:
# View all results (AUROC) by each params
list(zip(pipelineModel_gbt_hpo.avgMetrics, 
         pipelineModel_gbt_hpo.getEstimatorParamMaps()))

[(0.991749422965293,
  {Param(parent='GBTClassifier_8c9c74b6ce2d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='GBTClassifier_8c9c74b6ce2d', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32,
   Param(parent='GBTClassifier_8c9c74b6ce2d', name='maxIter', doc='max number of iterations (>= 0).'): 10}),
 (0.9915485855418229,
  {Param(parent='GBTClassifier_8c9c74b6ce2d', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5,
   Param(parent='GBTClassifier_8c9c74b6ce2d', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 64,
   Param(parent='GBT

In [ ]:
# Predict and find AUROC, test error and accuracy
prediction_gbt = pipelineModel_gbt_hpo.transform(testDF) 

gbt_auroc = evaluator_auroc.evaluate(prediction_gbt)
print('GridSearchCV: Gradient Boosted Trees')
print('Area under ROC curve: %g' % (gbt_auroc))
print('Test Error: %g ' % (1.0 - gbt_auroc))
print('Accuracy:', evaluator_acc.evaluate(prediction_gbt)) 

GridSearchCV: Gradient Boosted Trees
Area under ROC curve: 0.986468
Test Error: 0.013532 
Accuracy: 0.9858603889722596


In [ ]:
# Extract info from pipeline
pipelineModel_gbt_hpo.bestModel.stages[-1].extractParamMap()

{Param(parent='GBTClassifier_8c9c74b6ce2d', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='GBTClassifier_8c9c74b6ce2d', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='GBTClassifier_8c9c74b6ce2d', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for r

In [ ]:
# Find best model and params
bestPipeline = pipelineModel_gbt_hpo.bestModel
bestModel = bestPipeline.stages[-1]
bestParams = bestModel.extractParamMap()
print(bestPipeline)
print('\n')
print(bestModel)
print('\n')
print(bestParams)

PipelineModel_7aed6edf9218


GBTClassificationModel: uid = GBTClassifier_8c9c74b6ce2d, numTrees=10, numClasses=2, numFeatures=50


{Param(parent='GBTClassifier_8c9c74b6ce2d', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False, Param(parent='GBTClassifier_8c9c74b6ce2d', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10, Param(parent='GBTClassifier_8c9c74b6ce2d', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatic

In [ ]:
# Find best model and params
best_mod = pipelineModel_gbt_hpo.bestModel
param_dict = best_mod.stages[-1].extractParamMap()
sane_dict = {}
for k, v in param_dict.items():
  sane_dict[k.name] = v
sane_dict

{'cacheNodeIds': False,
 'checkpointInterval': 10,
 'featureSubsetStrategy': 'all',
 'featuresCol': 'unscaledFeatures',
 'impurity': 'variance',
 'labelCol': 'label',
 'leafCol': '',
 'lossType': 'logistic',
 'maxBins': 64,
 'maxDepth': 15,
 'maxIter': 10,
 'maxMemoryInMB': 256,
 'minInfoGain': 0.0,
 'minInstancesPerNode': 1,
 'minWeightFractionPerNode': 0.0,
 'predictionCol': 'prediction',
 'probabilityCol': 'probability',
 'rawPredictionCol': 'rawPrediction',
 'seed': 42,
 'stepSize': 0.1,
 'subsamplingRate': 1.0,
 'validationTol': 0.01}